Version 1.0.0

# Introduction

In this programming assignment we will illustrate a very severe data leakage, that can often be found in competitions, where the pairs of object should be scored, e.g. predict $1$ if two objects belong to the same class and $0$ otherwise. 

The data in this assignment is taken from a real competition, and the funniest thing is that *we will not use training set at all* and achieve almost 100% accuracy score! We will just exploit the leakage.

Now go through the notebook and complete the assignment.

In [1]:
import numpy as np
import pandas as pd 
import scipy.sparse

# Load the data

Let's load the test data. Note, that we don't have any training data here, just test data. Moreover, *we will not even use any features* of test objects. All we need to solve this task is the file with the indices for the pairs, that we need to compare.

Let's load the data with test indices.

In [2]:
test = pd.read_csv('../readonly/data_leakages_data/test_pairs.csv')
test.head(10)

,pairId,FirstId,SecondId
0,0,1427,8053
1,1,17044,7681
2,2,19237,20966
3,3,8005,20765
4,4,16837,599
5,5,3657,12504
6,6,2836,7582
7,7,6136,6111
8,8,23295,9817
9,9,6621,7672


For example, we can think that there is a test dataset of images, and each image is assigned a unique `Id` from $0$ to $N-1$ (N -- is the number of images). In the dataframe from above `FirstId` and `SecondId` point to these `Id`'s and define pairs, that we should compare: e.g. do both images in the pair belong to the same class or not. So, for example for the first row: if images with `Id=1427` and `Id=8053` belong to the same class, we should predict $1$, and $0$ otherwise. 

But in our case we don't really care about the images, and how exactly we compare the images (as long as comparator is binary).  

**We suggest you to try to solve the puzzle yourself first.** You need to submit a `.csv` file with columns `pairId` and `Prediction` to the grader. The number of submissions allowed is made pretty huge to let you explore the data without worries. The returned score should be very close to $1$.

**If you do not want to think much** -- scroll down and follow the instructions below.

# EDA and leakage intuition

As we already know, the key to discover data leakages is careful EDA. So let's start our work with some basic data exploration and build an intuition about the leakage.

First, check, how many different `id`s are there: concatenate `FirstId` and `SecondId` and print the number of unique elements. Also print minimum and maximum value for that vector.

In [ ]:
# YOUR CODE GOES HERE

# データ型確認
# IDの和をとる？→これだと同じ和になるIDの組み合わせが同一のものとみなされるので駄目
# 文字列としてIDを結合する。ただし、逆の組み合わせも同一と見なせるよう、大きい数字_小さい数字の順でID結合する
# IDの組み合わせの数を確認

In [3]:
# データの確認
test.describe()

,pairId,FirstId,SecondId
count,368550.000000,368550.000000,368550.000000
mean,184274.500000,10863.601118,11950.398882
std,106391.365192,7280.190939,7602.814820
min,0.000000,0.000000,0.000000
25%,92137.250000,4574.000000,5698.000000
50%,184274.500000,9886.000000,10512.000000
75%,276411.750000,16512.000000,18782.000000
max,368549.000000,26324.000000,26324.000000


In [4]:
# 大きい数字_小さい数字でID連結する関数
def merge_id(x):
    if x.FirstId >= x.SecondId:
        return x.FirstId_str + '_' + x.SecondId_str
    else:
        return x.SecondId_str + '_' + x.FirstId_str

In [5]:
test['FirstId_str'] = test['FirstId'].astype(str)
test['SecondId_str'] = test['SecondId'].astype(str)

In [6]:
%time
test['merge_id'] = test.apply(lambda x: merge_id(x), axis=1)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.54 µs


In [7]:
test.head()

,pairId,FirstId,SecondId,FirstId_str,SecondId_str,merge_id
0,0,1427,8053,1427,8053,8053_1427
1,1,17044,7681,17044,7681,17044_7681
2,2,19237,20966,19237,20966,20966_19237
3,3,8005,20765,8005,20765,20765_8005
4,4,16837,599,16837,599,16837_599


In [8]:
test.nunique()
# 368439個のユニークなIDの組み合わせがある

pairId          368550
FirstId          26325
SecondId         26310
FirstId_str      26325
SecondId_str     26310
merge_id        368439
dtype: int64

and then print how many pairs we need to classify (it is basically the number of rows in the test set)

In [9]:
# YOUR CODE GOES HERE
print('pairs we need to classify: ', len(test))

pairs we need to classify:  368550


Now print, how many distinct pairs it would be possible to create out of all "images" in the dataset?   

In [10]:
# YOUR CODE GOES HERE
print('distinct pairs we need to classify: ', 368439)

distinct pairs we need to classify:  368439


So the number of pairs we are given to classify is very very small compared to the total number of pairs. 

To exploit the leak we need to **assume (or prove)**, that the total number of positive pairs is small, compared to the total number of pairs. For example: think about an image dataset with $1000$ classes, $N$ images per class. Then if the task was to tell whether a pair of images belongs to the same class or not, we would have $1000\frac{N(N-1)}{2}$ positive pairs, while total number of pairs was $\frac{1000N(1000N - 1)}{2}$.

Another example: in [Quora competitition](https://www.kaggle.com/c/quora-question-pairs) the task was to classify whether a pair of qustions are duplicates of each other or not. Of course, total number of question pairs is very huge, while number of duplicates (positive pairs) is much much smaller.

Finally, let's get a fraction of pairs of class `1`. We just need to submit a constant prediction "all ones" and check the returned accuracy. Create a dataframe with columns `pairId` and `Prediction`, fill it and export it to `.csv` file. Then submit to grader and examine grader's output. 

In [11]:
# YOUR CODE GOES HERE
submission_ver0 = test.loc[:,['pairId']]
submission_ver0['Prediction'] = 1

submission_ver0.to_csv('submission_ver0.csv', index=False)

In [12]:
submission_ver0.head()

,pairId,Prediction
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [13]:
pwd

'/home/jovyan/work/Programming assignment, week 2: Data leakages'

# 全て1で提出した場合

accuracy = 0.5だった。  
つまり、提出物のうち半分は正解が0ということ

So, we assumed the total number of pairs is much higher than the number of positive pairs, but it is not the case for the test set. It means that the test set is constructed not by sampling random pairs, but with a specific sampling algorithm. Pairs of class `1` are oversampled.

Now think, how we can exploit this fact? What is the leak here? If you get it now, you may try to get to the final answer yourself, othewise you can follow the instructions below.   

# Building a magic feature

In this section we will build a magic feature, that will solve the problem almost perfectly. The instructions will lead you to the correct solution, but please, try to explain the purpose of the steps we do to yourself -- it is very important.

## Incidence matrix

First, we need to build an [incidence matrix](https://en.wikipedia.org/wiki/Incidence_matrix). You can think of pairs `(FirstId, SecondId)` as of edges in an undirected graph. 

The incidence matrix is a matrix of size `(maxId + 1, maxId + 1)`, where each row (column) `i` corresponds `i-th` `Id`. In this matrix we put the value `1` to the position `[i, j]`, if and only if a pair `(i, j)` or `(j, i)` is present in  a given set of pais `(FirstId, SecondId)`. All the other elements in the incidence matrix are zeros.   

**Important!** The incidence matrices are typically very very sparse (small number of non-zero values). At the same time incidence matrices are usually huge in terms of total number of elements, and it is **impossible to store them in memory in dense format**. But due to their sparsity incidence matrices **can be easily represented as sparse matrices**. If you are not familiar with sparse matrices, please see [wiki](https://en.wikipedia.org/wiki/Sparse_matrix) and [scipy.sparse reference](https://docs.scipy.org/doc/scipy/reference/sparse.html). Please, use any of `scipy.sparse` constructors to build incidence matrix. 

For example, you can use this constructor: `scipy.sparse.coo_matrix((data, (i, j)))`. We highly recommend to learn to use different `scipy.sparse` constuctors, and matrices types, but if you feel you don't want to use them, you can always build this matrix with a simple `for` loop. You will need first to create a matrix using `scipy.sparse.coo_matrix((M, N), [dtype])` with an appropriate shape `(M, N)` and then iterate through `(FirstId, SecondId)` pairs and fill corresponding elements in matrix with ones. 

**Note**, that the matrix should be symmetric and consist only of zeros and ones. It is a way to check yourself.

In [14]:
test.head()

,pairId,FirstId,SecondId,FirstId_str,SecondId_str,merge_id
0,0,1427,8053,1427,8053,8053_1427
1,1,17044,7681,17044,7681,17044_7681
2,2,19237,20966,19237,20966,20966_19237
3,3,8005,20765,8005,20765,20765_8005
4,4,16837,599,16837,599,16837_599


In [15]:
# make sparse matrix
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, lil_matrix
from numpy import array

# 26325*26325の疎行列を作成
# pairになっているものに1を代入する

# Sparse matrix の試し書き

In [16]:
# 疎行列を作成
mat_tmp = lil_matrix((5, 5))

In [17]:
# 非ゼロ要素を設定する
mat_tmp[0, 0] = 1
mat_tmp[0, 3] = 1
mat_tmp[2, 4] = 1

In [18]:
# 非ゼロ要素を設定する
for i in range(0, 5):
    for j in range(0, 5):
        if i == j:
            mat_tmp[i, j] = 1

In [19]:
print(mat_tmp)

  (0, 0)	1.0
  (0, 3)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (2, 4)	1.0
  (3, 3)	1.0
  (4, 4)	1.0


In [20]:
# coo matrixに変換
mat_tmp = mat_tmp.tocoo(copy = True)

In [21]:
mat_tmp.max()

1.0

In [22]:
mat_tmp.sum()

7.0

In [23]:
# csrへの変換
mat_tmp = mat_tmp.tocsr()

In [24]:
print(mat_tmp)

  (0, 0)	1.0
  (0, 3)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (2, 4)	1.0
  (3, 3)	1.0
  (4, 4)	1.0


In [25]:
# 列のindexを表示
mat_tmp.indices

array([0, 3, 1, 2, 4, 3, 4], dtype=int32)

In [26]:
# 行のindexを表示
mat_tmp.indptr

array([0, 2, 3, 5, 6, 7], dtype=int32)

In [27]:
print(mat_tmp.todense())

[[ 1.  0.  0.  1.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]


In [28]:
print(mat_tmp[np.array([0, 2, 2])])

  (0, 3)	1.0
  (0, 0)	1.0
  (1, 4)	1.0
  (1, 2)	1.0
  (2, 4)	1.0
  (2, 2)	1.0


In [29]:
print(mat_tmp[np.array([0, 2, 2])].todense())

[[ 1.  0.  0.  1.  0.]
 [ 0.  0.  1.  0.  1.]
 [ 0.  0.  1.  0.  1.]]


# Incidence matrix の作成

In [30]:
print('First: ', test['FirstId'].max())
print('Second: ', test['SecondId'].max())

First:  26324
Second:  26324


In [31]:
# 疎行列を作成
mat = lil_matrix((test['FirstId'].max() + 1, test['SecondId'].max() + 1))

In [32]:
test.head()

,pairId,FirstId,SecondId,FirstId_str,SecondId_str,merge_id
0,0,1427,8053,1427,8053,8053_1427
1,1,17044,7681,17044,7681,17044_7681
2,2,19237,20966,19237,20966,20966_19237
3,3,8005,20765,8005,20765,20765_8005
4,4,16837,599,16837,599,16837_599


In [33]:
%time
for i in range(0, len(test)):
    first = test['FirstId'].values[i]
    second = test['SecondId'].values[i]
    mat[first, second] = 1
    mat[second, first] = 1

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [34]:
# coo matrixに変換
mat_coo = mat.tocoo(copy = True)

In [35]:
mat_coo

<26325x26325 sparse matrix of type '<class 'numpy.float64'>'
	with 736872 stored elements in COOrdinate format>

In [36]:
inc_mat = mat_coo # YOUR CODE GOES HERE (but probably you will need to write few more lines before)

# Sanity checks
assert inc_mat.max() == 1
assert inc_mat.sum() == 736872

It is convenient to have matrix in `csr` format eventually.

In [37]:
# csrに変換
inc_mat = inc_mat.tocsr()

# sparse matrixのスライス 試し書き
https://cmdlinetips.com/2019/07/how-to-slice-rows-and-columns-of-sparse-matrix-in-python/

In [38]:
from scipy import sparse
from scipy import stats

In [39]:
# coo matrixを作成
A = sparse.random(5, 5,
                  density=0.5,
                  data_rvs=stats.poisson(10, loc=10).rvs)

In [40]:
print(A)

  (3, 4)	21.0
  (4, 0)	17.0
  (2, 2)	18.0
  (1, 1)	11.0
  (1, 0)	18.0
  (0, 2)	19.0
  (4, 2)	23.0
  (4, 1)	20.0
  (2, 1)	18.0
  (4, 4)	18.0
  (1, 3)	19.0
  (0, 4)	23.0


In [41]:
print(A.todense())

[[  0.   0.  19.   0.  23.]
 [ 18.  11.   0.  19.   0.]
 [  0.  18.  18.   0.   0.]
 [  0.   0.   0.   0.  21.]
 [ 17.  20.  23.   0.  18.]]


## Now build the magic feature

Why did we build the incidence matrix? We can think of the rows in this matix as of representations for the objects. `i-th` row is a representation for an object with `Id = i`. Then, to measure similarity between two objects we can measure similarity between their representations. And we will see, that such representations are very good.

Now select the rows from the incidence matrix, that correspond to `test.FirstId`'s, and `test.SecondId`'s.

In [42]:
# Note, scipy goes crazy if a matrix is indexed with pandas' series. 
# So do not forget to convert `pd.series` to `np.array`
# These lines should normally run very quickly 

# test.FirstId、test.SecondIdそれぞれについて、inc_matからスライスで切り出す
# なお、1つのFirstIdに複数のSecondIdが対応しており、1つのFirstIdで複数行持つので、test.FirstIdのidは重複している
# そのため、切り出した後のmatrixの行数はもとのmatrix(idの重複がない)より多くなる
rows_FirstId   = inc_mat[np.array(test['FirstId'])]# YOUR CODE GOES HERE
rows_SecondId  = inc_mat[np.array(test['SecondId'])]# YOUR CODE GOES HERE

In [43]:
inc_mat.shape

(26325, 26325)

In [44]:
rows_FirstId.shape

(368550, 26325)

In [45]:
rows_SecondId.shape

(368550, 26325)

Our magic feature will be the *dot product* between representations of a pair of objects. Dot product can be regarded as similarity measure -- for our non-negative representations the dot product is close to 0 when the representations are different, and is huge, when representations are similar. 

Now compute dot product between corresponding rows in `rows_FirstId` and `rows_SecondId` matrices.

# multiplyのテスト

In [46]:
mat_a = mat_tmp = lil_matrix((3, 2))
mat_b = mat_tmp = lil_matrix((1, 2))

mat_a[0,0] = 1; mat_a[0,1] = 2; mat_a[1,0] = -1; mat_a[2,0] = 3; mat_a[2,1] = 1
mat_b[0,0] = 7; mat_b[0,1] = 1

print(mat_a.todense())
print(mat_b.todense())

[[ 1.  2.]
 [-1.  0.]
 [ 3.  1.]]
[[ 7.  1.]]


In [47]:
mat_a.shape

(3, 2)

In [48]:
mat_b.shape

(1, 2)

In [49]:
mat_a = mat_a.tocsr()

In [50]:
print(mat_a.todense())

[[ 1.  2.]
 [-1.  0.]
 [ 3.  1.]]


In [51]:
mat_b = mat_b.tocsr()

In [52]:
mat_c = mat_a.multiply(mat_b)
print(mat_c.todense())

[[  7.   2.]
 [ -7.   0.]
 [ 21.   1.]]


In [53]:
mat_c.sum(axis=1)

matrix([[  9.],
        [ -7.],
        [ 22.]])

# magic futureをbuildしてみる

In [54]:
f_tmp = rows_FirstId.multiply(rows_SecondId).sum(axis = 1)

In [55]:
f_tmp.shape

(368550, 1)

In [56]:
# Note, that in order to do pointwise multiplication in scipy.sparse you need to use function `multiply`
# regular `*` corresponds to matrix-matrix multiplication

f = f_tmp# YOUR CODE GOES HERE

# Sanity check
assert f.shape == (368550, 1)

That is it! **We've built our magic feature.** 

# From magic feature to binary predictions

But how do we convert this feature into binary predictions? We do not have a train set to learn a model, but we have a piece of information about test set: the baseline accuracy score that you got, when submitting constant. And we also have a very strong considerations about the data generative process, so probably we will be fine even without a training set. 

We may try to choose a thresold, and set the predictions to 1, if the feature value `f` is higer than the threshold, and 0 otherwise. What threshold would you choose? 

How do we find a right threshold? Let's first examine this feature: print frequencies (or counts) of each value in the feature `f`.

In [57]:
print(f)

[[ 20.]
 [ 14.]
 [ 20.]
 ..., 
 [ 14.]
 [ 14.]
 [ 14.]]


In [58]:
# fをarray変換してflattenしてから、ユニークな値を確認
np.unique(np.array(f).ravel())

array([ 14.,  15.,  19.,  20.,  21.,  28.,  35.])

In [59]:
data_frame = pd.DataFrame(np.array(f).ravel(), columns=['feature'])

In [61]:
data_frame.head()

,feature
0,20.0
1,14.0
2,20.0
3,20.0
4,14.0


In [60]:
len(data_frame)

368550

In [62]:
data_frame.groupby(['feature']).size()

feature
14.0    183279
15.0       852
19.0       546
20.0    183799
21.0         6
28.0        54
35.0        14
dtype: int64

In [63]:
183279/368550

0.4972975172975173

In [64]:
# For example use `np.unique` function, check for flags

print(np.unique(np.array(f).ravel())) # YOUR CODE GOES HERE

[ 14.  15.  19.  20.  21.  28.  35.]


Do you see how this feature clusters the pairs? Maybe you can guess a good threshold by looking at the values? 

In fact, in other situations it can be not that obvious, but in general to pick a threshold you only need to remember the score of your baseline submission and use this information. Do you understand why and how?  

Choose a threshold below: 

In [65]:
pred = f > 14 # SET THRESHOLD HERE

In [66]:
pred

matrix([[ True],
        [False],
        [ True],
        ..., 
        [False],
        [False],
        [False]], dtype=bool)

# Finally, let's create a submission

In [67]:
submission = test.loc[:,['pairId']]
submission['Prediction'] = pred.astype(int)

submission.to_csv('submission.csv', index=False)

In [68]:
pd.value_counts(submission['Prediction'])

1    185271
0    183279
Name: Prediction, dtype: int64

Now submit it to the grader! It is not possible to submit directly from this notebook, as we need to submit a `csv` file, not a single number (limitation of Coursera platform). 

To download `submission.csv` file that you've just produced <a href='./submission.csv'>click here</a> (if the link opens in browser, right-click on it and shoose "Save link as"). Then go to [assignment page](https://www.coursera.org/learn/competitive-data-science/programming/KsASv/data-leakages/submission) and submit your `.csv` file in 'My submission' tab.


If you did everything right, the score should be very high.

**Finally:** try to explain to yourself, why the whole thing worked out. In fact, there is no magic in this feature, and the idea to use rows in the incidence matrix can be intuitively justified.

# Bonus

Interestingly, it is not the only leak in this dataset. There is another totally different way to get almost 100% accuracy. Try to find it!